In [211]:
import numpy as np
import random
import math


In [212]:
# la bound
# lb bound
# ht bound


In [213]:
def gen_deadline(exec_time, period):
    # [a,b]
    b = 1.2 * period
    if(exec_time < 10):
        a = exec_time
    elif(exec_time < 100 and exec_time >= 10):
        a = 2*exec_time
    elif(exec_time < 1000 and exec_time >= 100):
        a = 3*exec_time
    else:
        a = 4*exec_time

    return a+random.random()*(b-a)


def gen_tasksets(utilizations, periods):
    """
    Take a list of task utilization sets and a list of task period sets and
    return a list of couples (c, p) sets. The computation times are truncated
    at a precision of 10^-10 to avoid floating point precision errors.
    Args:
        - `utilization`: The list of task utilization sets. For example::
            [[0.3, 0.4, 0.8], [0.1, 0.9, 0.5]]
        - `periods`: The list of task period sets. For examples::
            [[100, 50, 1000], [200, 500, 10]]
    Returns:
        For the above example, it returns::
            [[(30.0, 100), (20.0, 50), (800.0, 1000)],
             [(20.0, 200), (450.0, 500), (5.0, 10)]]
    """
    def trunc(x, p):
        return int(x * 10 ** p) / float(10 ** p)
    [[print(ui) for ui, pi in zip(us, ps)]
            for us, ps in zip(utilizations, periods)]
    return np.array([[[trunc(ui * pi, 6), trunc(pi, 6), trunc(gen_deadline(trunc(ui * pi, 6), trunc(pi, 6)), 6)] for ui, pi in zip(us, ps)]
            for us, ps in zip(utilizations, periods)])


In [214]:
def custom_period(n, nsets,max_,min_=0):
	max_power = np.log(max_)


	intervals = np.arange(0,np.floor(max_power)+1)
	intervals = np.append(intervals, max_power)
	#print(intervals)
	task_pp = ((n-1)//(len(intervals)-1))
	
	task_lf = np.mod(n-1, len(intervals)-1).astype(int)
	#print(task_lf,task_pp)
	periods = np.zeros((nsets, n))
	for i in range(nsets):
		for j in range(len(intervals)-1):
			if(j<task_lf):
				periods[i][j*task_pp+j:(j+1)*task_pp+j+1] = np.random.uniform(low=np.exp(intervals[j]), high=np.exp(intervals[j+1]), size=task_pp+1)
			else:
				periods[i][j*task_pp+task_lf:(j+1)*task_pp+task_lf] = np.random.uniform(low=np.exp(intervals[j]), high=np.exp(intervals[j+1]), size=task_pp)
	periods[0:nsets,-1] =max_ 
	return periods


In [215]:
custom_period(14,2,100)

array([[  2.10765532,   1.92216629,   2.02721094,   6.95718807,
          3.08060647,   7.19715638,  19.26062991,  17.53285512,
          9.45821757,  50.62044335,  51.67280797,  84.84321758,
         70.66386538, 100.        ],
       [  1.40174473,   2.2577439 ,   1.86582113,   5.1294107 ,
          7.28893078,   4.23344035,  16.2065652 ,  15.9343966 ,
         18.58344573,  23.71503106,  20.27039615,  67.04156965,
         72.14869789, 100.        ]])

In [216]:
def UUniFastDiscard(n, u, nsets):
    sets = []
    while len(sets) < nsets:
        # Classic UUniFast algorithm:
        utilizations = []
        sumU = u
        for i in range(1, n):
            nextSumU = sumU * random.random() ** (1.0 / (n - i))
            utilizations.append(sumU - nextSumU)
            sumU = nextSumU
        utilizations.append(sumU)

        # If no task utilization exceeds 1:
        if all(ut <= 1 for ut in utilizations):
            sets.append(utilizations)

    return sets

    
def gen_uunifastdiscard(nsets, u, n):
    """
    The UUniFast algorithm was proposed by Bini for generating task
    utilizations on uniprocessor architectures.
    The UUniFast-Discard algorithm extends it to multiprocessor by
    discarding task sets containing any utilization that exceeds 1.
    This algorithm is easy and widely used. However, it suffers from very
    long computation times when n is close to u. Stafford's algorithm is
    faster.
    Args:
        - `n`: The number of tasks in a task set.
        - `u`: Total utilization of the task set.
        - `nsets`: Number of sets to generate.
    Returns `nsets` of `n` task utilizations.
    """
    return UUniFastDiscard(n, u, nsets)

In [217]:
def get_taskset(nsets):
	u = 0.9
	n = 1000
	utilizations = gen_uunifastdiscard(nsets, u, n)
	periods = custom_period(n, nsets, 1000, 10)
	return gen_tasksets(utilizations, periods)

	

In [218]:
k = get_taskset(1)
z = k[0]
print(k)
print(z[:,2])
#gen_uunifastdiscard(u=0.9, n=5, nsets=1)

2.1434661677699474e-05
0.0007239124691348353
0.00032993865669983524
0.0015309498013864609
0.000757704879536969
0.0004876401674803743
0.0008589597996652154
0.0010154247830320706
0.0018859270944900919
0.00029659674304416406
0.0008978107605820407
0.0010426603441999749
0.00013728493108355888
0.0003264177455016837
0.0015940124388160726
0.0026324421973981105
0.0005305859487295672
0.00160369091224688
0.0005299930204664127
0.0007506508982059179
0.0020052722818620916
0.0009307524170694137
0.0034540417061159623
0.0004501296117340736
0.00027596769105187047
0.00026488161960736
0.000686667812109576
0.001497346028329627
0.0014248537559602426
0.0005558574958564044
0.0006941678232508242
0.001124213987839573
0.0011824934403833476
0.0021913190546870576
0.0007829988390711806
0.001656212593668327
0.00044775385434814297
0.000291990782074647
0.0008837721750389882
0.0030096928510563448
0.0014976275193627764
0.0009020423067310768
0.0006019487517362476
0.001791023169294248
0.00020401102683309524
0.000853438357

In [219]:
def procDemand_func(taskset,t):
    deadlines = taskset[:,2]
    periods = taskset[:,1]
    utilizations = taskset[:,0]/taskset[:,1]#wcet/periods
	#print(deadlines, periods, utilizations)
    total_utilization = np.sum(utilizations)

    task_done   = (1+np.floor((t-deadlines)/(periods)))*taskset[:,0]#wcet

    task_done[task_done<0] = 0

    h_t = np.sum(task_done)

    return h_t

In [220]:
def La_bound(taskset):
	"""
	Compute La calculations for the La bound
	
	"""
	
	deadlines = taskset[:,2]
	periods = taskset[:,1]
	utilizations = taskset[:,0]/taskset[:,1]
	#print(deadlines, periods, utilizations)
	total_utilization = np.sum(utilizations)
	#print(np.sum((periods-deadlines)*utilizations)/(1-total_utilization))
	La = np.maximum(np.max(deadlines),np.sum((periods-deadlines)*utilizations)/(1-total_utilization))
	return La


In [221]:
bound = La_bound(k[0])
print(k[0])
print(bound)

print(procDemand_func(k[0],900))

[[4.80000000e-05 2.25992600e+00 2.52177300e+00]
 [1.47300000e-03 2.03519900e+00 5.65554000e-01]
 [5.37000000e-04 1.62890400e+00 1.34357100e+00]
 ...
 [6.25791000e-01 6.26839834e+02 2.68895817e+02]
 [3.87149000e-01 9.63288766e+02 1.05762860e+02]
 [1.89058000e-01 1.00000000e+03 1.06105827e+03]]
1081.987866
788.934661


In [222]:
def Lb_bound(taskset):
    return 100


In [223]:
def qpa_algorithm(taskset):
    """
    Compute h(t) calculations for the QPA algorithm

    """
    taskset_schedulable = False
    deadlines = taskset[:,2]
    periods = taskset[:,1]
    utilizations = taskset[:,0]/taskset[:,1]#wcet/periods


    l_a = La_bound(taskset)
    l_b = Lb_bound(taskset)
    min_bound = min(l_a,l_b)
    min_deadline = np.min(deadlines)
    max_deadline = np.max(deadlines[deadlines<min_bound])

    t = max_deadline
    count =1
    while(procDemand_func(taskset,t) <= t and procDemand_func(taskset,t)>min_deadline):
        if(procDemand_func(taskset,t) <= t):
            t = procDemand_func(taskset,t)
        else:
            t = np.max(deadlines[deadlines<t])
        count+=1
    

    if(procDemand_func(taskset,t)<=min_deadline):
        taskset_schedulable = True

    return count
    




	

In [225]:
taskset_paper = np.array([[6000,18000,31000],[2000,9000,9800],[1000,12000,17000],[90,3000,4200],[8,78,96],[2,16,12],[10,120,280],[26,160,660]])

In [224]:
qpa_algorithm(k[0])

KeyboardInterrupt: 

In [226]:
print(La_bound(taskset_paper))

35560.03904490162
